In [5]:
import sys
import os

# Add parent directory to path
from src.db_connection import DatabaseConnection


db = DatabaseConnection()
con = db.get_connection()


cursor = con.cursor()
cursor.execute("select * from bronze.covid;")

print(cursor.fetchall())

ModuleNotFoundError: No module named 'src'

In [ ]:
from datetime import date


date.today()

datetime.date(2025, 11, 9)

In [ ]:
print(db_host, db_password)

localhost covid


In [4]:
import pandas as pd

# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.db_connection import DatabaseConnection


db = DatabaseConnection()

df = db.read_dataframe_from_db("SELECT * FROM BRONZE.covid;")


df.head(10)

,fips,admin2,province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,combined_key,incident_rate,case_fatality_ratio,ingested_at
0,NaN,None,None,Afghanistan,2021-01-02 05:22:33,33.93911,67.709953,52513,2201,41727,8585,Afghanistan,134.896578,4.191343,2021-01-01
1,NaN,None,None,Albania,2021-01-02 05:22:33,41.15330,20.168300,58316,1181,33634,23501,Albania,2026.409062,2.025173,2021-01-01
2,NaN,None,None,Algeria,2021-01-02 05:22:33,28.03390,1.659600,99897,2762,67395,29740,Algeria,227.809861,2.764848,2021-01-01
3,NaN,None,None,Andorra,2021-01-02 05:22:33,42.50630,1.521800,8117,84,7463,570,Andorra,10505.403482,1.034865,2021-01-01
4,NaN,None,None,Angola,2021-01-02 05:22:33,-11.20270,17.873900,17568,405,11146,6017,Angola,53.452981,2.305328,2021-01-01
5,NaN,None,None,Antigua and Barbuda,2021-01-02 05:22:33,17.06080,-61.796400,159,5,148,6,Antigua and Barbuda,162.364186,3.144654,2021-01-01
6,NaN,None,None,Argentina,2021-01-02 05:22:33,-38.41610,-63.616700,1629594,43319,1426676,159599,Argentina,3605.633332,2.658269,2021-01-01
7,NaN,None,None,Armenia,2021-01-02 05:22:33,40.06910,45.038200,159738,2828,143355,13555,Armenia,5390.664389,1.770399,2021-01-01
8,NaN,None,Australian Capital Territory,Australia,2021-01-02 05:22:33,-35.47350,149.012400,118,3,114,1,"Australian Capital Territory, Australia",27.563653,2.542373,2021-01-01
9,NaN,None,New South Wales,Australia,2021-01-02 05:22:33,-33.86880,151.209300,4947,54,0,4893,"New South Wales, Australia",60.938655,1.091571,2021-01-01


In [4]:
from datetime import datetime


now = datetime.now().date()

str(now)

'2025-11-15'

In [ ]:
from dateutil.parser import parse


dt = parse("12-5-19")
print(dt.strftime("%d-%m-%Y"))

05-12-2019


In [4]:
"01-01-2025.csv".split(".")

['01-01-2025', 'csv']

In [5]:
file_date = parse("21-01|2025").strftime("%d-%m-%Y")

ParserError: Unknown string format: 21-01|2025

In [ ]:
from dateutil.parser import parse
import re


def get_date_from_file_name(file_name: str) -> str:
    """
    Return the date of the file by processing the filename and extract the date from the filename.
    in date format of day-month-year
    """

    content = file_name.split(".")

    if len(content) != 2:
        raise ValueError("The file name not as expected DD-MM-YYYY")

    file_date = parse(content[0]).strftime("%d-%m-%Y")

    file_name = file_date + "." + content[1]

    pattern = r"^\d{2}-\d{2}-\d{4}\.csv$"

    if not re.match(pattern, file_name):
        raise ValueError("The file name format not as expected.")

    return file_date


x = get_date_from_file_name("01-01-2025.csv")
print

ParserError: wqeqweqwewq

In [8]:
def null_df():
    return pd.DataFrame({"c1": [1, 2, None], "c2": [None, 1.2, 5]})


null_df()

,c1,c2
0,1.0,NaN
1,2.0,1.2
2,NaN,5.0


In [19]:
import sys
import os

# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.db_connection import DatabaseConnection


def read_row_data_from_bronze() -> pd.DataFrame:
    """
    Read the row data from the bronze layer and load it into a pandas dataframe for processing.
    Raise an error if can't load the table
    """
    try:
        query = """
                    WITH latest_ingestion AS (
                        SELECT MAX(ingested_at) as max_date 
                        FROM bronze.covid
                    )
                    SELECT * 
                    FROM bronze.covid b
                    CROSS JOIN latest_ingestion l
                    WHERE b.ingested_at = l.max_date;
                """
        with DatabaseConnection() as db_connection:
            row_data = db_connection.read_dataframe_from_db(query)
            row_data = row_data.drop(columns=["max_date"])
            return row_data
    except Exception as e:
        raise type(e)(f"Can't load the the data from database: {e}")


def drop_columns(df: pd.DataFrame, columns_list: list) -> pd.DataFrame:
    """
    Drop list of columns from the dataframe and raise an error if can't drop the columns.
    """
    if df.columns.empty:
        raise ValueError("The dataframe has no columns to delete.")

    try:
        return df.drop(columns=columns_list)

    except Exception as e:
        raise type(e)(f"Can't drop the columns , not exists: {e}")


def drop_null_values(df: pd.DataFrame, columns_list: list) -> pd.DataFrame:
    """
    drop rows that have null values from the dataframe, taking list of columns to drop the rows if this column have null value.
    """
    try:
        return df.dropna(subset=columns_list)
    except Exception as e:
        raise type(e)(f"Can't drop the rows the columns not exists , {e}")


def drop_negative_values(df: pd.DataFrame, columns_list: list) -> pd.DataFrame:
    """
    drop rows that have negative values from the dataframe, taking list of columns to drop the rows if this column have negative value
    """
    try:
        mask = (df[columns_list] >= 0).all(axis=1)

        return df[mask]
    except Exception as e:
        raise type(e)(f"Can't drop the row the columns not exists : {e}")


def format_locations(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    format the locations columns to be in title format to be in a standard form.
    """

    formated_df = df.copy()

    try:
        for col in columns:
            formated_df[col] = formated_df[col].str.strip().str.lower().str.title()

        return formated_df
    except Exception as e:
        raise type(e)(f"The columns not exists: {e}")


def standardize_null_values(df: pd.DataFrame, columns_list: list) -> pd.DataFrame:
    """Standardize null value representations."""
    df_clean = df.copy()
    null_representations = ["None", "Unknown", "", " ", "N/A", "null"]

    for column in columns_list:
        df_clean[column] = df_clean[column].replace(null_representations, "None")

    return df_clean


def load_cleansed_data_into_silver(cleansed_data: pd.DataFrame):
    """
    Storing the transformed data into the database.
    """
    try:
        with DatabaseConnection() as db_connection:
            db_connection.load_dataframe_into_db(cleansed_data, "silver", "covid")
    except Exception as e:
        raise type(e)(f"Can't load the transformed data into the database: {e}")


def run_transformation() -> dict:
    """
    Combine all the transformation logic to make it easy for orchestration.
    and return dictionary with information about no.loaded , tranformed, stored and status.
    """

    row_data = read_row_data_from_bronze()
    columns = ["fips", "admin2", "combined_key", "last_update", "lat", "long_"]
    transformed_data = drop_columns(row_data, columns)

    columns = (
        "country_region",
        "confirmed",
        "deaths",
        "recovered",
        "active",
        "incident_rate",
        "case_fatality_ratio",
    )
    transformed_data = drop_null_values(transformed_data, columns)

    columns = ["deaths", "confirmed", "recovered", "active", "incident_rate"]
    transformed_data = drop_negative_values(transformed_data, columns)

    columns_locations = ["province_state", "country_region"]
    transformed_data = format_locations(transformed_data, columns_locations)

    columns_list = ["province_state"]
    transformed_data = standardize_null_values(transformed_data, columns_list)

    load_cleansed_data_into_silver(transformed_data)

    return {
        "loaded_data": len(row_data),
        "rows_transformed": len(transformed_data),
        "status": "success",
    }

In [20]:
df = pd.DataFrame(
    {"col1": ["A", None, "X"], "col2": ["A", "B", "C"], "col3": [False, True, None]}
)


# Given df with null values
columns_to_check = ["col1", "col2", "col3"]

# When


df = drop_null_values(df, columns_to_check)

df

,col1,col2,col3
0,A,A,False


In [6]:
df = pd.DataFrame({"c1": [-1, -1, -1], "c2": [-2, -2, -2], "c3": [-3, -3, -3]})

df = drop_negative_values(df, ["c1", "c2", "c3"])
df

,c1,c2,c3


In [7]:
x = pd.DataFrame({"c1": [], "c2": [], "c3": []})
x

,c1,c2,c3


In [8]:
if (x == df).all().all():
    print("Y")

Y


In [10]:
print("data     data".title())

Data     Data


In [21]:
import pandas as pd

df = pd.DataFrame({"c1": ["data", "None"], "c2": ["N/A", "null"]})
df.head(2)
df = standardize_null_values(df, ["c1", "c2"])
df

,c1,c2
0,data,None
1,None,None


In [7]:
from typing import Optional


def func(x: Optional[str] = None):
    y = x or "DEV"
    return y


func("test")

'test'

In [9]:
from dotenv import load_dotenv
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
load_dotenv()  # loads .env file


print(os.environ["APP_ENV"])
print(os.environ["TEST_DB_HOST"])

DEV
localhost


In [ ]:
print("IO")

: 

In [1]:
print("X")

X


In [2]:
2+2

4